# KnightSeq 2.0
## Isaac Burmingham

The following is a bokeh web application that does preliminary cuts to input star cluster data.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import astropy
from bokeh import *
from bokeh.io import output_notebook,show,output_file
from bokeh.plotting import figure,save,show,curdoc
from bokeh.models import ColumnDataSource, CustomJS,Row
from bokeh.transform import jitter
from bokeh.models.tools import *
from bokeh.models.widgets import *
from bokeh.layouts import row,column
from bokeh.models.widgets import Button
%matplotlib inline
output_notebook()

Loading BokehJS ...

In [2]:
testdata = pd.read_csv("/Users/Isaac/knight/7789aaron.stand",sep="\s+")
fiddata = pd.read_csv("/Users/Isaac/knight/7789aaron.fid",sep="\s+")

The columns orginally input as 99.9999 are replaced with NaN so that they are not plotted. The color combinations are all recalculated.

In [3]:
testdata["U"] = testdata["U"].replace(99.999,np.nan) 
testdata["B"] = testdata["B"].replace(99.999,np.nan) 
testdata["V"] = testdata["V"].replace(99.999,np.nan)
testdata["R"] = testdata["R"].replace(99.999,np.nan) 
testdata["I"] = testdata["I"].replace(99.999,np.nan)
testdata["U-B"] = testdata["U"]-testdata["B"]
testdata["U-V"] = testdata["U"]-testdata["V"]
testdata["U-R"] = testdata["U"]-testdata["R"]
testdata["U-I"] = testdata["U"]-testdata["I"]
testdata["B-V"] = testdata["B"]-testdata["V"]
testdata["B-R"] = testdata["B"]-testdata["R"]
testdata["B-I"] = testdata["B"]-testdata["I"]
testdata["V-R"] = testdata["V"]-testdata["R"]
testdata["V-I"] = testdata["V"]-testdata["I"]
testdata["R-I"] = testdata["R"]-testdata["I"]

In [4]:
cols = ['U','B','V','R','I','U-B','U-R','U-I','B-V','B-R','B-I','V-R','V-I','R-I']
dffilters = testdata[cols]

Primary code is written below. Since it is defined as a function, the show command line must be run. Currently, it is limited by the tools that Bokeh has implemented itself. 

In [5]:
from bokeh.models.widgets import Dropdown

def modify_doc(doc):
    select_tools = ['lasso_select', 'tap', 'reset','box_zoom',
                'wheel_zoom','pan','undo']
    # B-V
    
    BVV = ColumnDataSource(data=dict(x=testdata['B-V'],y=testdata['V']))
    
    data = dffilters
    source = ColumnDataSource(data={
    'x'       : data['B-V'],
    'y'       : data['V']})
    
    plot = figure(title='Knight Seq',plot_height=400, plot_width=700,x_range=(0,2),y_range=(20,12),tools=select_tools)

    # Add circle glyphs to the plot
    plot.circle(x='x', y='y',nonselection_alpha=0.5,selection_color='red',selection_alpha=0.8, source=source)

    def update_plot(attr, old, new):
        # Read the current value off the drop down
        
        x = x_select.value
        y = y_select.value
        # Label axes of plot
        plot.xaxis.axis_label = x
        plot.yaxis.axis_label = y
        # Set new_data
        new_data = {
            'x' : data[x],
            'y' : data[y],
        }
        # Assign new_data to source.data
        source.data = new_data
        

        # Set the range of all axes
        plot.x_range.start = min(data[x])
        plot.x_range.end = max(data[x])
        plot.y_range.start = max(data[y])
        plot.y_range.end = min(data[y])

    # Create a dropdown Select widget for the x data: x_select
    x_select = Select(title="X Filters", options=['U','V','B','R','I','B-V','U-B','U-R','U-I','B-R','B-I','V-R','V-I','R-I'], value='B-V')


    # Attach the update_plot callback to the 'value' property of x_select
    x_select.on_change('value', update_plot)

    # Create a dropdown Select widget for the y data: y_select
    y_select = Select(title="Y Filters", options=['U','V','B','R','I','B-V','U-B','U-R','U-I','B-R','B-I','V-R','V-I','R-I'], value='V')

    # Attach the update_plot callback to the 'value' property of y_select
    y_select.on_change('value', update_plot)
    
    s2 = ColumnDataSource(data = dict(x = [], y = []))
#     p2 = figure(plot_width = 400, plot_height = 400, x_range = (0, 2), y_range = (20, 12), tools = "", title = "Watch Here")
#     p2.circle('x', 'y', source = s2, alpha = 0.6)

    source.selected.js_on_change('indices', CustomJS(args=dict(source=source, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = source.data;
        d2 = {'x': [], 'y': []}
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.data = d2  """)) # change so that it doesn't push values, outputs master bool
    

    def get_values():
        print(s2.data)
    
    def to_dataf():
        global df
        df = s2.to_df()
        df['bool'] = False
        print("Length",len(df)) # prints length of selected
        #print(df)
        return df

    #button = Button(label = "Print Selected")
    button2= Button(label= "Get Data")
    #button.on_click(get_values)
    button2.on_click(to_dataf)
    
    doc.add_root(column(x_select,y_select,Row(plot),button2))
    # Use selection as a delete

When running the show command, make sure your notebook_url starts with "localhost:8888" this might be a different number.

# Menu

Below is the interactive plot. There are two dropdowns from which you can select your x axis and y axis. Do so will change the axis and also change the zoom.
Toolbar (in order):
>1: Pan Tool <br>
2: Lasso Tool - This tool is a draw selection tool. It is currently programmed to select the stars that are to be DELETED, not kept. The stars that are selected will turn a bright red color. In order to make multiple selections, you must press SHIFT. <br>
3: Box Zoom Tool - Zoom tool that the user draws a box to zoom in on. <br>
4: Scroll Zoom Tool - This is a scroll zoom, place the mouse where you would like to zoom and simply scroll in or out. If you put your mouse over the x or y axis, it will change the scale of the axis. <br>
5: Tap Tool: Tap a point in order to select it. <br>
6: Reset Button: Tap to reset the entire selection. <br>
7: Undo Button: Tap to undo recent selection. <br>

Once you are happy with the selection, press the "Get Data" button. A length will display showing that the button was clicked. 

In [6]:
show(modify_doc,notebook_url="http://localhost:8888")

In [8]:
testdata['master'] = df['bool']
testdata['master'] = testdata['master'].fillna(True)

In [9]:
finaldf = testdata[testdata['master'] == True] # Keep the non-selected
finaldf

,#starid,x(pix),y(pix),U,sig(SNR),sig(disp),B,sig(SNR).1,sig(disp).1,V,...,U-V,U-R,U-I,B-V,B-R,B-I,V-R,V-I,R-I,master
2142,2990,221.468,755.430,18.233,0.024,0.049,17.814,0.010,0.056,16.807,...,1.426,2.061,2.672,1.007,1.642,2.253,0.635,1.246,0.611,True
2143,2991,1758.458,755.994,17.597,0.014,0.024,17.479,0.008,0.028,16.683,...,0.914,1.403,1.912,0.796,1.285,1.794,0.489,0.998,0.509,True
2144,2995,1204.221,756.031,18.711,0.028,0.080,18.473,0.012,0.041,17.591,...,1.120,1.610,2.177,0.882,1.372,1.939,0.490,1.057,0.567,True
2145,2994,958.078,756.059,19.398,0.069,0.141,18.836,0.017,0.036,17.843,...,1.555,2.158,2.721,0.993,1.596,2.159,0.603,1.166,0.563,True
2146,2997,1471.667,756.151,15.468,0.004,0.005,15.350,0.004,0.004,14.703,...,0.765,1.147,1.571,0.647,1.029,1.453,0.382,0.806,0.424,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,5000816,971.428,2048.806,NaN,9.999,9.999,20.111,0.051,0.222,19.091,...,NaN,NaN,NaN,1.020,NaN,NaN,NaN,NaN,NaN,True
6691,5000817,1281.539,2049.121,NaN,9.999,9.999,18.400,0.017,0.010,17.388,...,NaN,NaN,NaN,1.012,1.586,NaN,0.574,NaN,NaN,True
6692,5000818,1884.369,2049.410,NaN,9.999,9.999,19.804,0.039,0.072,18.535,...,NaN,NaN,NaN,1.269,1.890,2.421,0.621,1.152,0.531,True
6693,5000819,959.957,2049.681,NaN,9.999,9.999,13.425,0.008,0.000,12.934,...,NaN,NaN,NaN,0.491,NaN,NaN,NaN,NaN,NaN,True


To export to a csv, you can use the finaldf.to_csv('filepath') command.